In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import matplotlib.pyplot as plt
import tqdm
from torchvision.utils import save_image
import itertools
from torchvision import models

In [2]:
class ImagePairDataset(Dataset):
    def __init__(self, folder1, folder2, pairs, transform=None, enhance_transform_1 = None, 
                 enhance_transform_2 = None, enhance_transform_3 = None):
        self.folder1 = folder1
        self.folder2 = folder2
        self.pairs = pairs
        self.transform = transform
        self.enhance_transform_1 = enhance_transform_1
        self.enhance_transform_2 = enhance_transform_2
        self.enhance_transform_3 = enhance_transform_3
        self.image_pairs = self.read_image_pairs()

    def read_image_pairs(self):
      image_pairs = []
      for image_pair in tqdm.tqdm(self.pairs):
        img1_path = os.path.join(self.folder1, image_pair[0])
        img2_path = os.path.join(self.folder2, image_pair[1])
        img1 = Image.open(img1_path).convert("RGB")
        img2 = Image.open(img2_path).convert("RGB")
        if self.transform:
            img9 = self.transform(img1)
            img10 = self.transform(img2)
            image_pairs.append((img9, img10))
        if self.enhance_transform_1:
            img3 = self.enhance_transform_1(img1)
            img4 = self.enhance_transform_1(img2)
            image_pairs.append((img3, img4))
        if self.enhance_transform_2:
            img5 = self.enhance_transform_2(img1)
            img6 = self.enhance_transform_2(img2)
            image_pairs.append((img5, img6))
        if self.enhance_transform_3:
            img7 = self.enhance_transform_3(img1)
            img8 = self.enhance_transform_3(img2)
            image_pairs.append((img7, img8))
            
      return image_pairs
        
    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
      return self.image_pairs[idx][0], self.image_pairs[idx][1]

In [3]:
transform = transforms.Compose([
    transforms.Resize((128, 256)),
    # transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 归一化
])

In [4]:
digital_dir = "/root/autodl-tmp/Kodak_gold_200/Kodak_gold_200/digital"
film_dir = "/root/autodl-tmp/Kodak_gold_200/Kodak_gold_200/film"

In [5]:
def make_pairs(image1_dir, imgage2_dir):
    # Get the list of files in both folders
    images1 = sorted(os.listdir(image1_dir))
    images2 = sorted(os.listdir(imgage2_dir))
    
    print(len(images1))
    print(len(images2))

    # Ensure the number of files match
    if len(images1) != len(images2):
        raise ValueError("The two folders must have the same number of images.")

    # Create pairs of images (file1, file2)
    pairs = list(zip(images1, images2))
    
    return pairs

In [6]:
pairs = make_pairs(digital_dir, film_dir)

1002
1002


In [7]:
batch_size = 64

In [8]:
# Create datasets
train_dataset = ImagePairDataset(digital_dir, film_dir, pairs, transform = transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

100%|██████████| 1002/1002 [00:39<00:00, 25.36it/s]


In [ ]:
class ResNetGenerator(nn.Module):
    def __init__(self, input_nc=3, output_nc=3, pretrained=True):
        super(ResNetGenerator, self).__init__()

        # Loading pre-trained ResNet-18
        resnet = models.resnet18()
        resnet.load_state_dict(torch.load("./resnet18-5c106cde.pth"))
        
        # Encoder: Get the output of each layer of ResNet and construct U-Net style skip connections
        self.input_conv = nn.Sequential(
            resnet.conv1,   # Output: 64 channels, H/2 x W/2
            resnet.bn1,
            resnet.relu
        )
        self.maxpool = resnet.maxpool   # H/4 x W/4
        self.encoder1 = resnet.layer1   # Output: 64 channels, H/4 x W/4
        self.encoder2 = resnet.layer2   # Output: 128 channels, H/8 x W/8
        self.encoder3 = resnet.layer3   # Output: 256 channels, H/16 x W/16
        self.encoder4 = resnet.layer4   # Output: 512 channels, H/32 x W/32

        # Decoder: Use upsampling (bilinear interpolation) + convolution to restore details and fuse with skip connection features
        self.up1 = self._up_block(512, 256)  # H/32 -> H/16
        self.up2 = self._up_block(512, 128)  # 256(from up1)+256(encoder3) -> H/16 -> H/8
        self.up3 = self._up_block(256, 64)   # 128(from up2)+128(encoder2) -> H/8 -> H/4
        self.up4 = self._up_block(128, 64)   # 64(from up3)+64(encoder1) -> H/4 -> H/2
        self.up5 = self._up_block(128, 64)   # 64(from up4)+64(from input_conv) -> H/2 -> H

        self.final_conv = nn.Conv2d(64, output_nc, kernel_size=1)

        # Added perceptual loss module (VGG16 feature extraction)
        self.vgg = models.vgg16()
        self.vgg.load_state_dict(torch.load("./vgg16-397923af.pth"))
        self.vgg  = self.vgg.features[:16].eval()
        for param in self.vgg.parameters():
            param.requires_grad = False  # Freeze VGG weights

    def _up_block(self, in_channels, out_channels):
        """
        Upsampling block: upsampling first (bilinear), then convolution + ReLU
        """
        block = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        return block

    def add_film_grain(self, x, noise_strength=0.1):
        """
        Add Gaussian noise to the image Tensor to simulate film grain effect.
        """
        batch_size, channels, height, width = x.shape
        mean = 0
        sigma = noise_strength * 255  # Control the standard deviation of noise
        
        # Generate noise and overlay it on the image
        noise = torch.normal(mean, sigma, size=(batch_size, channels, height, width), dtype=torch.float32, device=x.device)
        noisy_image = x + noise  # Adding Noise
        return noisy_image

    def perceptual_loss(self, real, fake):
        """
        Compute perceptual loss, based on VGG feature extraction.
        """
        real_features = self.vgg(real)
        fake_features = self.vgg(fake)
        return torch.nn.functional.mse_loss(real_features, fake_features)

    def forward(self, x):
        h = x.shape[2]
        w = x.shape[3]

        # Encoder part
        x0 = self.input_conv(x)       # x0: [B, 64, H/2, W/2]
        x1 = self.maxpool(x0)         # x1: [B, 64, H/4, W/4]
        x1 = self.encoder1(x1)        # x1: [B, 64, H/4, W/4]
        x2 = self.encoder2(x1)        # x2: [B, 128, H/8, W/8]
        x3 = self.encoder3(x2)        # x3: [B, 256, H/16, W/16]
        x4 = self.encoder4(x3)        # x4: [B, 512, H/32, W/32]

        # Decoder part: gradually upsample and concatenate with the corresponding layer features of the encoder
        d1 = self.up1(x4)             # d1: [B, 256, H/16, W/16]
        d1 = torch.cat([d1, x3], dim=1)  # [B, 256+256=512, H/16, W/16]

        d2 = self.up2(d1)             # d2: [B, 128, H/8, W/8]
        d2 = torch.cat([d2, x2], dim=1)  # [B, 128+128=256, H/8, W/8]

        d3 = self.up3(d2)             # d3: [B, 64, H/4, W/4]
        d3 = torch.cat([d3, x1], dim=1)  # [B, 64+64=128, H/4, W/4]

        d4 = self.up4(d3)             # d4: [B, 64, H/2, W/2]
        d4 = torch.cat([d4, x0], dim=1)  # [B, 64+64=128, H/2, W/2]

        d5 = self.up5(d4)             # d5: [B, 64, H, W]

        out = self.final_conv(d5)     # out: [B, output_nc, H, W]
        out = torch.tanh(out)        
        
        out = self.add_film_grain(out, noise_strength=0.0002)

        return out, self.perceptual_loss(x, out)

In [10]:
# ============================
# 3. Defining the Discriminator (PatchGAN)
# ============================
class Discriminator(nn.Module):
    def __init__(self, input_nc):
        """
        Using 70×70 PatchGAN discriminator
        """
        super(Discriminator, self).__init__()
        model = [
            nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        in_channels = 64
        out_channels = in_channels * 2
        # Add several layers of convolution
        for _ in range(3):
            model += [
                nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1),
                nn.InstanceNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True)
            ]
            in_channels = out_channels
            out_channels = in_channels * 2

        # The last convolution layer
        model += [
            nn.Conv2d(in_channels, 1, kernel_size=4, padding=1)
        ]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [11]:

# ============================
# 4. Define the CycleGAN model class (add AMP support)
# ============================
class CycleGAN:
    def __init__(self, device, G_AB = "", G_BA = "", D_A = "", D_B = ""):
        self.device = device

        # Initialize two generators: G_AB (digital → film), G_BA (film → digital)
        self.G_AB = ResNetGenerator(3, 3).to(device)
        if G_AB != "":
            self.G_AB.load_state_dict(torch.load(G_AB))
        self.G_BA = ResNetGenerator(3, 3).to(device)
        if G_BA != "":
            self.G_BA.load_state_dict(torch.load(G_BA))
        # Initialize two discriminators: D_A (discriminates real digital images) and D_B (discriminates real film images)
        self.D_A = Discriminator(3).to(device)
        if D_A != "":
            self.D_A.load_state_dict(torch.load(D_A))
        self.D_B = Discriminator(3).to(device)
        if D_B != "":
            self.D_B.load_state_dict(torch.load(D_B))
        # Define loss function: adversarial loss, cycle consistency loss, identity loss
        self.criterion_GAN = nn.MSELoss().to(device)
        self.criterion_cycle = nn.L1Loss().to(device)
        self.criterion_identity = nn.L1Loss().to(device)

        # Optimizer (two generators share one optimizer)
        self.optimizer_G = optim.Adam(itertools.chain(self.G_AB.parameters(), self.G_BA.parameters()),
                                    lr=0.0002, betas=(0.5, 0.999))
        self.optimizer_D_A = optim.Adam(self.D_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
        self.optimizer_D_B = optim.Adam(self.D_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

        # Creating an AMP GradScaler object
        self.scaler_G = torch.amp.GradScaler(device=device)
        self.scaler_D_A = torch.amp.GradScaler(device=device)
        self.scaler_D_B = torch.amp.GradScaler(device=device)

    def set_input(self, real_A, real_B):
        self.real_A = real_A.to(self.device)
        self.real_B = real_B.to(self.device)

    def forward(self):
        # A→B→A
        self.fake_B, _ = self.G_AB(self.real_A)
        self.rec_A, _ = self.G_BA(self.fake_B)
        # B→A→B
        self.fake_A, _ = self.G_BA(self.real_B)
        self.rec_B, _ = self.G_AB(self.fake_A)

    def backward_G(self):
        # Identity loss: requires the generator to remain invariant on target domain images
        self.idt_A, _ = self.G_BA(self.real_A)
        self.loss_idt_A = self.criterion_identity(self.idt_A, self.real_A) * 5.0 + _
        self.idt_B, _ = self.G_AB(self.real_B)
        self.loss_idt_B = self.criterion_identity(self.idt_B, self.real_B) * 5.0 + _

        # Fighting Losses
        pred_fake_B = self.D_B(self.fake_B)
        target_real = torch.ones_like(pred_fake_B, device=self.device)
        loss_GAN_AB = self.criterion_GAN(pred_fake_B, target_real)

        pred_fake_A = self.D_A(self.fake_A)
        target_real = torch.ones_like(pred_fake_A, device=self.device)
        loss_GAN_BA = self.criterion_GAN(pred_fake_A, target_real)

        # Cycle consistency loss
        loss_cycle_A = self.criterion_cycle(self.rec_A, self.real_A) * 10.0
        loss_cycle_B = self.criterion_cycle(self.rec_B, self.real_B) * 10.0

        # Total generator loss (only saved, no backward call)
        self.loss_G = self.loss_idt_A + self.loss_idt_B + loss_GAN_AB + loss_GAN_BA + loss_cycle_A + loss_cycle_B

    def backward_D_basic(self, netD, real, fake):
        target_real = torch.ones_like(netD(real), device=self.device)
        target_fake = torch.zeros_like(netD(fake.detach()), device=self.device)
        loss_real = self.criterion_GAN(netD(real), target_real)
        loss_fake = self.criterion_GAN(netD(fake.detach()), target_fake)
        loss_D = (loss_real + loss_fake) * 0.5
        return loss_D

    def backward_D_A(self):
        self.loss_D_A = self.backward_D_basic(self.D_A, self.real_A, self.fake_A)

    def backward_D_B(self):
        self.loss_D_B = self.backward_D_basic(self.D_B, self.real_B, self.fake_B)

    def optimize_parameters(self):
        # --------------------------
        # Update Generator
        # --------------------------
        self.optimizer_G.zero_grad()
        with torch.cuda.amp.autocast():
            self.forward()       # Generate fake_B, fake_A, rec_A, rec_B
            self.backward_G()    # Calculate self.loss_G
        self.scaler_G.scale(self.loss_G).backward()
        self.scaler_G.step(self.optimizer_G)
        self.scaler_G.update()

        # --------------------------
        # Update the discriminator D_A
        # --------------------------
        self.optimizer_D_A.zero_grad()
        with torch.cuda.amp.autocast():
            self.backward_D_A()  # Calculate self.loss_D_A
        self.scaler_D_A.scale(self.loss_D_A).backward()
        self.scaler_D_A.step(self.optimizer_D_A)
        self.scaler_D_A.update()

        # --------------------------
        # Update the discriminator D_B
        # --------------------------
        self.optimizer_D_B.zero_grad()
        with torch.cuda.amp.autocast():
            self.backward_D_B()  # Calculate self.loss_D_B
        self.scaler_D_B.scale(self.loss_D_B).backward()
        self.scaler_D_B.step(self.optimizer_D_B)
        self.scaler_D_B.update()

In [12]:
# ============================
# 5. Training loop function
# ============================
def train(cyclegan, dataloader, num_epochs=200, save_interval=10, pre_epoch = 0):
    for epoch in tqdm.tqdm(range(num_epochs)):
        for i, (data_A, data_B) in enumerate(dataloader):
            real_A = data_A
            real_B = data_B
            cyclegan.set_input(real_A, real_B)
            cyclegan.optimize_parameters()

        # Save output results and models regularly
        if (epoch + 1) % save_interval == 0:
            os.makedirs("output", exist_ok=True)
            with torch.no_grad():
                fake_B, _ = cyclegan.G_AB(real_A.cuda())
                fake_A, _ = cyclegan.G_BA(real_B.cuda())
            save_image(fake_B, f"output/Kodak_gold_200_3/fake_B_epoch_{epoch+1+pre_epoch}.png", normalize=True)
            save_image(fake_A, f"output/Kodak_gold_200_3/fake_A_epoch_{epoch+1+pre_epoch}.png", normalize=True)
            torch.save(cyclegan.G_AB.state_dict(), f"output/Kodak_gold_200_3/G_AB_epoch_{epoch+1+pre_epoch}.pth")
            torch.save(cyclegan.G_BA.state_dict(), f"output/Kodak_gold_200_3/G_BA_epoch_{epoch+1+pre_epoch}.pth")
            torch.save(cyclegan.D_A.state_dict(), f"output/Kodak_gold_200_3/D_A_epoch_{epoch+1+pre_epoch}.pth")
            torch.save(cyclegan.D_B.state_dict(), f"output/Kodak_gold_200_3/D_B_epoch_{epoch+1+pre_epoch}.pth")


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
cyclegan = CycleGAN(device)

/tmp/ipykernel_1193/2430728245.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load("./resnet18-5c106cde.pth"))
/tmp/ipykernel_1193/24307282

In [33]:
cyclegan = CycleGAN(device, G_AB="output/Kodak_gold_200_3/G_AB_epoch_200.pth", 
                    G_BA="output/Kodak_gold_200_3/G_BA_epoch_200.pth", 
                    D_A="output/Kodak_gold_200_3/D_A_epoch_200.pth", 
                    D_B="output/Kodak_gold_200_3/D_B_epoch_200.pth")

/tmp/ipykernel_1186/2430728245.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load("./resnet18-5c106cde.pth"))
/tmp/ipykernel_1186/24307282

In [ ]:
train(cyclegan, train_loader, num_epochs=1000, pre_epoch = 0, save_interval=100)

  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_1193/2374480379.py:92: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/tmp/ipykernel_1193/2374480379.py:103: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/tmp/ipykernel_1193/2374480379.py:113: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
  1%|          | 6/1000 [01:01<2:48:56, 10.20s/it]

In [ ]:
os.system("/usr/bin/shutdown")